In [16]:
import math

In [ ]:
def compute_slabthickness (assumedslabthickness):
    recommendedslabthickness = L / 33 * m_to_mm
    if recommendedslabthickness >= assumedslabthickness:
        slabthickness = recommendedslabthickness
    else:
        slabthickness = assumedslabthickness
    return slabthickness

def compute_slabdepth (slabthickness):
    slabdepth = slabthickness - 25.5  # mm
    return slabdepth

def ultimateuniformload (deadload, liveload):
    wu = 1.2 * deadload + 1.6 * liveload # kN/m
    result = wu
    return result

def ultimatemoment ():
    neg_mu = wu * math.pow(clearspan, 2) / 11 # kN-m
    pos_mu = wu * math.pow(clearspan, 2) / 8  # kN-m
    results = neg_mu, pos_mu
    return results

In [23]:
ratio = fys / (0.85*fc)

def check_rho (ratio, fys, beta1, width, tensionarea, slabdepth):
    m_to_mm = 1000
    rhobal = beta1 * 600 / (ratio * (600 + fys))
    rhomax = 0.75 * rhobal #TODO change rhomax to new code formula
    rhoact = tensionarea / (width * m_to_mm * slabdepth)
    if rhoact <= rhomax:
        rho = rhoact
    else:
        rho = rhomax
    return rho

def depthOfCompressionBlock (rho, ratio, slabdepth):
    a = rho * ratio * slabdepth # mm

def momentcapacity ():
    mucap = 0.9 * tensionarea * fys * (slabdepth - a /2) / 1e6 # N-mm to kN-m
    if mucap > pos_mu:
        result1 = "No additional reinforcement is required for bottom pan"
    result = result1, mucap
    return result

## To be continued

In [ ]:
def ultimatemomenttop ():
    

In [17]:
""" DESIGN CRITERIA """

# MATERIAL STRENGTH
thickness = 1 # steel deck thickness in mm
fc = 21 # concrete compressive strength in Mpa
fy = 275.8 # minimum yield strength of reinforcing bars in Mpa
fys = 275 # yield strength of steel deck in Mpa
clearspan = 2.5 # clear span between supports in m
assumedslabthickness = 125 # in mm
beta1 = 0.85

# LOADINGS
width = 1 # in m
psf_to_kpa = 0.0478802
slabweight = 61.46 * psf_to_kpa
finish_toppings = 35 * psf_to_kpa
ceiling_utilities = 5 * psf_to_kpa
superimposed_DL = slabweight + finish_toppings + ceiling_utilities
superimposed_LL = 40 * psf_to_kpa
deadload = superimposed_DL * width # kN/m
liveload = superimposed_LL * width # kN/m

# INPUT FROM MANUAL
tensionarea = 1083 # steel deck tension area in mm2

In [ ]:
def checkunshoredspan(deckprofile, thickness, span, wetload, construlive):
    """
    Checks if the chosen deck profile and thickness can support the unshored span.
    
    Parameters:
      deckprofile : str          # deck profile (e.g., "typea", "typeb")
      thickness   : float        # deck sheet thickness in mm (e.g., 1.20)
      span        : float        # unshored span in meters (e.g., 3.2)
      wetload     : float        # wet concrete load in kN/m² (e.g., 2.5)
      construlive : float        # construction live load in kN/m² (e.g., 1.0)
    
    Uses a lookup table (hypothetical data) for maximum span in meters.
    """
    # Example lookup table: key = (deckprofile, thickness)
    # Value = maximum unshored span in meters
    unshoredtable = {
        ("typea", 1.20): 3.0,
        ("typea", 1.40): 3.5,
        ("typeb", 1.20): 3.2,
        ("typeb", 1.60): 4.0
    }
    
    key = (deckprofile, thickness)
    if key not in unshoredtable:
        return (False, "ng: no unshored span data for " + deckprofile + ", " + str(thickness) + " #mm")
    
    maxspan = unshoredtable[key]  # max span in m from table
    # Total construction load (kN/m²) is the sum of wet concrete and live loads.
    totalload = wetload + construlive  # kN/m²
    
    # (In a full design, the load effect would be calculated and compared to moment capacities.)
    if span <= maxspan:
        return (True, "ok: unshored span " + str(span) + " m <= " + str(maxspan) + " m limit for " + deckprofile)
    else:
        return (False, "ng: unshored span " + str(span) + " m exceeds " + str(maxspan) + " m limit for " + deckprofile)

def checkcompositecapacity(deckprofile, thickness, slabthickness, fydeck, superimposed):
    """
    Checks the composite deck-slab bending capacity.
    
    Parameters:
      deckprofile   : str    # deck profile (e.g., "typea", "typeb")
      thickness     : float  # deck sheet thickness in mm (e.g., 1.20)
      slabthickness : float  # total slab thickness in mm (e.g., 120)
      fydeck        : float  # yield strength of deck steel in MPa (e.g., 320)
      superimposed  : float  # superimposed load in kN/m² (e.g., 5.0)
    
    Hypothetical example:
      Base capacity is taken from a table (in kN/m²).
      For each 10 mm above 100 mm slab thickness, add 0.5 kN/m².
      A factor is applied if fydeck > 300 MPa.
    """
    capacitytable = {
        ("typea", 1.20): 8.0,
        ("typea", 1.40): 10.0,
        ("typeb", 1.20): 9.5,
        ("typeb", 1.60): 12.0
    }
    
    key = (deckprofile, thickness)
    if key not in capacitytable:
        return (False, "ng: no capacity data for " + deckprofile + ", " + str(thickness))
    
    basecap = capacitytable[key]  # in kN/m²
    if slabthickness > 100:
        # For each 10 mm above 100 mm, add 0.5 kN/m² capacity
        extrathick = slabthickness - 100  # mm
        addcap = (extrathick // 10) * 0.5  # kN/m²
    else:
        addcap = 0.0
    fyfactor = 1.0
    if fydeck > 300:
        fyfactor = 1.1  # arbitrary bump for higher steel strength
    
    totalcap = (basecap + addcap) * fyfactor  # in kN/m²
    
    if superimposed <= totalcap:
        return (True, "ok: composite capacity ~" + str(round(totalcap,2)) + " kN/m² >= " + str(superimposed) + " kN/m² required")
    else:
        return (False, "ng: demanded " + str(superimposed) + " kN/m² > capacity " + str(round(totalcap,2)) + " kN/m²")

def checkwebcrippling(deckprofile, thickness, bearingwidth, reaction):
    """
    Checks web crippling capacity at supports.
    
    Parameters:
      deckprofile : str   # deck profile (e.g., "typea", "typeb")
      thickness   : float # deck sheet thickness in mm (e.g., 1.20)
      bearingwidth: float # bearing width on support in mm (e.g., 80)
      reaction  : float   # reaction at support in kN per meter (e.g., 45.0)
    
    Uses a simplified lookup table (in kN/m) for base web crippling capacity,
    and applies a bearing factor if the bearing width exceeds 60 mm.
    """
    criptable = {
        ("typea", 1.20): 40,  # base capacity in kN/m
        ("typea", 1.40): 50,
        ("typeb", 1.20): 45,
        ("typeb", 1.60): 60
    }
    
    key = (deckprofile, thickness)
    if key not in criptable:
        return (False, "ng: no web crippling data for " + deckprofile + ", " + str(thickness))
    
    basecrip = criptable[key]  # in kN/m
    bearingfactor = 1.0
    if bearingwidth > 60:
        bearingfactor = 1.1  # if bearing width is greater than 60 mm, capacity increases slightly
    
    capacity = basecrip * bearingfactor  # in kN/m
    
    if reaction <= capacity:
        return (True, "ok: web crippling capacity ~" + str(capacity) + " kN/m >= reaction " + str(reaction) + " kN/m")
    else:
        return (False, "ng: reaction " + str(reaction) + " kN/m > capacity " + str(capacity) + " kN/m")

def checkmaterials(fydeck, fcconc):
    """
    Checks if the material strengths are acceptable.
    
    Parameters:
      fydeck : float   # yield strength of deck steel in MPa (e.g., 320)
      fcconc : float   # concrete compressive strength in MPa (e.g., 27)
    
    Requirements (example):
      fydeck must be at least 250 MPa and fcconc at least 20 MPa.
    """
    if fydeck < 250:
        return (False, "ng: deck steel yield strength (" + str(fydeck) + " MPa) is below minimum 250 MPa")
    if fcconc < 20:
        return (False, "ng: concrete compressive strength (" + str(fcconc) + " MPa) is below 20 MPa")
    return (True, "ok: material strengths are acceptable")

def main():
    print("=== floor deck design (metric units) ===")
    
    # ---------------------------
    # SAMPLE INPUTS (all metric)
    # ---------------------------
    # Material strengths
    fydeck = 320      # MPa (e.g., yield strength of steel deck)
    fcconc = 27       # MPa (e.g., concrete compressive strength)
    
    # Deck geometry
    deckprofile = "typea"  # example deck profile ("typea", "typeb", etc.)
    thickness = 1.20       # mm (deck sheet thickness)
    
    # Slab geometry
    slabthickness = 120    # mm (total composite slab thickness)
    
    # Span and bearing details
    span = 3.2             # m (unshored span)
    bearingwidth = 80      # mm (bearing width at support)
    
    # Loads (in metric units)
    wetload = 2.5          # kN/m² (wet concrete load)
    construlive = 1.0      # kN/m² (construction live load)
    superimposed = 5.0     # kN/m² (superimposed load, e.g., finishes, live load)
    
    # Reaction at support
    reaction = 45.0        # kN/m (reaction from analysis)
    
    # ---------------------------
    # CHECKS
    # ---------------------------
    
    print("\n--- checking material strengths ---")
    matok, matmsg = checkmaterials(fydeck, fcconc)
    print(matmsg)  # e.g., "ok: material strengths are acceptable"
    
    print("\n--- checking unshored span ---")
    spanok, spanmsg = checkunshoredspan(deckprofile, thickness, span, wetload, construlive)
    print(spanmsg)
    
    print("\n--- checking composite capacity ---")
    compok, compmsg = checkcompositecapacity(deckprofile, thickness, slabthickness, fydeck, superimposed)
    print(compmsg)
    
    print("\n--- checking web crippling ---")
    cripok, cripmsg = checkwebcrippling(deckprofile, thickness, bearingwidth, reaction)
    print(cripmsg)
    
    print("\n=== summary of checks ===")
    if matok and spanok and compok and cripok:
        print("all demonstration checks passed. deck selection is acceptable for these sample inputs.")
    else:
        print("one or more checks failed. see details above.")

if __name__ == "__main__":
    main()

=== floor deck design (metric units) ===

--- checking material strengths ---
ok: material strengths are acceptable

--- checking unshored span ---
ng: unshored span 3.2 m exceeds 3.0 m limit for typea

--- checking composite capacity ---
ok: composite capacity ~9.9 kN/m² >= 5.0 kN/m² required

--- checking web crippling ---
ng: reaction 45.0 kN/m > capacity 44.0 kN/m

=== summary of checks ===
one or more checks failed. see details above.


In [ ]:
def knton(val):
    # converts kilonewtons to newtons using a multiplier of 1000
    return val * 1000

def checkunshoredspan(deckprofile, gauge, spanft, wetloadpsf, constructionlivepsf):
    """
    Checks if the chosen deck profile and gauge can support the unshored span.
    Loads (in psf) are combined from wet concrete and construction live load.
    Uses a simple lookup table (hypothetical data) for maximum span (ft).
    """
    # Example lookup table: keys are (deckprofile, gauge) with max span in ft
    unshoredtable = {
        ("2wh36", 22): 10,
        ("2wh36", 20): 11,
        ("2wh36", 18): 12,
        ("3wxh36", 22): 11,
        ("3wxh36", 20): 12,
        ("3wxh36", 18): 13
    }
    totalload = wetloadpsf + constructionlivepsf
    key = (deckprofile, gauge)
    if key not in unshoredtable:
        return (False, "no data for " + deckprofile + " gauge " + str(gauge))
    maxspan = unshoredtable[key]
    if spanft <= maxspan:
        return (True, "ok: " + deckprofile + " gauge " + str(gauge) + " can handle " + str(spanft) + " ft unshored span")
    else:
        return (False, "ng: " + deckprofile + " gauge " + str(gauge) + " limited to ~" + str(maxspan) + " ft, but " + str(spanft) + " ft was requested")

def checkcompositecapacity(deckprofile, gauge, slabthicknessin, fydeck, superimposedpsf):
    """
    Checks the composite deck-slab bending capacity using a hypothetical capacity table.
    For each inch above 4 inches slab thickness, adds 10 psf capacity.
    """
    capacitytable = {
        ("2wh36", 22): 60,
        ("2wh36", 20): 80,
        ("2wh36", 18): 100,
        ("3wxh36", 22): 70,
        ("3wxh36", 20): 90,
        ("3wxh36", 18): 120
    }
    key = (deckprofile, gauge)
    if key not in capacitytable:
        return (False, "no composite capacity data for " + deckprofile + " gauge " + str(gauge))
    basecapacity = capacitytable[key]
    if slabthicknessin > 4:
        thicknessfactor = (slabthicknessin - 4) * 10
    else:
        thicknessfactor = 0
    capacitypsf = basecapacity + thicknessfactor
    if superimposedpsf <= capacitypsf:
        return (True, "composite capacity ok: can support " + str(superimposedpsf) + " psf (limit ~" + str(capacitypsf) + " psf)")
    else:
        return (False, "composite capacity ng: demanded " + str(superimposedpsf) + " psf > capacity " + str(capacitypsf) + " psf")

def checkreactionswebcrippling(deckprofile, gauge, bearinglengthin, reactionplf):
    """
    Checks if the deck webs can handle the support reaction without crippling.
    Uses a simplified check with a base capacity (lbs per in) lookup.
    """
    # Convert reaction from lbs/ft to lbs/in; using integer division since no floats are allowed.
    reactionpli = reactionplf // 12
    basecripple = {22: 150, 20: 200, 18: 300, 16: 400}
    if gauge not in basecripple:
        return (False, "no web-cripple data for gauge " + str(gauge))
    capacitypli = basecripple[gauge]
    totalcapacity = capacitypli * bearinglengthin
    required = reactionpli * bearinglengthin
    if required <= totalcapacity:
        return (True, "web crippling ok: capacity ~" + str(totalcapacity) + " lbs >= required ~" + str(required) + " lbs")
    else:
        return (False, "web crippling ng: capacity ~" + str(totalcapacity) + " lbs < required ~" + str(required) + " lbs")

def main():
    print("=== floor deck design checker ===")
    print("sample demonstration of basic checks\n")
    
    # Sample inputs (all values are provided as integers or strings)
    deckprofile = "2wh36"   # example deck profile; acceptable values: "2wh36", "3wxh36"
    gauge = 20              # deck gauge, e.g. 22, 20, 18, or 16
    spanft = 10             # unshored span in feet
    wetloadpsf = 60         # approximate wet concrete load in psf
    constructionlivepsf = 20  # construction live load in psf
    superimposedpsf = 50    # superimposed load in psf
    slabthicknessin = 5     # total slab thickness in inches
    
    # For demonstration purposes, assume:
    fydeck = 33             # yield strength of deck steel (ksi)
    bearinglengthin = 2     # bearing length on support in inches
    reactionplf = 800       # reaction at support in lbs/ft
    
    print("\n--- checking unshored span ---")
    spanok, spanmsg = checkunshoredspan(deckprofile, gauge, spanft, wetloadpsf, constructionlivepsf)
    print(spanmsg)
    
    print("\n--- checking composite capacity ---")
    compok, compmsg = checkcompositecapacity(deckprofile, gauge, slabthicknessin, fydeck, superimposedpsf)
    print(compmsg)
    
    print("\n--- checking web crippling reactions ---")
    webok, webmsg = checkreactionswebcrippling(deckprofile, gauge, bearinglengthin, reactionplf)
    print(webmsg)
    
    print("\n=== summary of checks ===")
    if spanok and compok and webok:
        print("all demonstration checks passed. deck selection looks ok.")
    else:
        print("one or more checks failed or need revision. see messages above.")

if __name__ == "__main__":
    main()

=== floor deck design checker ===
sample demonstration of basic checks


--- checking unshored span ---
ok: 2wh36 gauge 20 can handle 10 ft unshored span

--- checking composite capacity ---
composite capacity ok: can support 50 psf (limit ~90 psf)

--- checking web crippling reactions ---
web crippling ok: capacity ~400 lbs >= required ~132 lbs

=== summary of checks ===
all demonstration checks passed. deck selection looks ok.
